# Retrieval

In [1]:
!pip install langchain_community

## 검색 할 파일 불러오기

### txt 파일

In [6]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./langchain/docs/docs/get_started/quickstart.mdx")

In [7]:
docs = loader.load()

In [8]:
len(docs)

1

In [9]:
docs

[Document(page_content='# Quickstart\n\nIn this quickstart we\'ll show you how to:\n- Get setup with LangChain, LangSmith and LangServe\n- Use the most basic and common components of LangChain: prompt templates, models, and output parsers\n- Use LangChain Expression Language, the protocol that LangChain is built on and which facilitates component chaining\n- Build a simple application with LangChain\n- Trace your application with LangSmith\n- Serve your application with LangServe\n\nThat\'s a fair amount to cover! Let\'s dive in.\n\n## Setup\n\n### Jupyter Notebook\n\nThis guide (and most of the other guides in the documentation) use [Jupyter notebooks](https://jupyter.org/) and assume the reader is as well. Jupyter notebooks are perfect for learning how to work with LLM systems because often times things can go wrong (unexpected output, API down, etc) and going through guides in an interactive environment is a great way to better understand them.\n\nYou do not NEED to go through the g

### 디렉토리 파일들

In [10]:
!pip install unstructured "unstructured[md]"

In [11]:
from langchain_community.document_loaders import DirectoryLoader

In [12]:
loader = DirectoryLoader('./langchain/docs/docs/', glob="**/*.md")

In [13]:
docs = loader.load()

In [14]:
len(docs)

19

In [17]:
docs[2]

Document(page_content='sidebar_class_name: hidden\n\nLangSmith\n\nLangSmith helps you trace and evaluate your language model applications and intelligent agents to help you\nmove from prototype to production.\n\nCheck out the interactive walkthrough to get started.\n\nFor more information, please refer to the LangSmith documentation.\n\nFor tutorials and other end-to-end examples demonstrating ways to integrate LangSmith in your workflow,\ncheck out the LangSmith Cookbook. Some of the guides therein include:\n\nLeveraging user feedback in your JS application (link).\n\nBuilding an automated feedback pipeline (link).\n\nHow to evaluate and audit your RAG workflows (link).\n\nHow to fine-tune an LLM on real usage data (link).\n\nHow to use the LangChain Hub to version your prompts (link)', metadata={'source': 'langchain/docs/docs/langsmith/index.md'})

### PDF

In [18]:
!pip install pypdf

In [19]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./langchain/libs/community/tests/examples/layout-parser-paper.pdf")
pages = loader.load_and_split()

In [20]:
len(pages)

16

In [22]:
pages[1]

Document(page_content='2 Z. Shen et al.\n37], layout detection [ 38,22], table detection [ 26], and scene text detection [ 4].\nA generalized learning-based framework dramatically reduces the need for the\nmanual speciﬁcation of complicated rules, which is the status quo with traditional\nmethods. DL has the potential to transform DIA pipelines and beneﬁt a broad\nspectrum of large-scale document digitization projects.\nHowever, there are several practical diﬃculties for taking advantages of re-\ncent advances in DL-based methods: 1) DL models are notoriously convoluted\nfor reuse and extension. Existing models are developed using distinct frame-\nworks like TensorFlow [ 1] or PyTorch [ 24], and the high-level parameters can\nbe obfuscated by implementation details [ 8]. It can be a time-consuming and\nfrustrating experience to debug, reproduce, and adapt existing models for DIA,\nand many researchers who would beneﬁt the most from using these methods lack\nthe technical background to 

## 검색하기

In [23]:
!pip install tiktoken faiss-cpu

In [45]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [25]:
loader = PyPDFLoader("./data/소개팅_나무위키.pdf")

In [26]:
pages = loader.load_and_split()

In [27]:
len(pages)

18

In [33]:
pages[3]

Document(page_content='24. 1. 7. 오후  1:32 소개팅  - 나무위키\nhttps://namu.wiki/w/ 소개팅 4/18\uf3d0\n\uf3d01. 아무리  퀸카라  하더라도  여중 , 여고 , 여대에  직장마저  여초  직장을  갖게  된다면  현실적으로  주변에서  자연스럽게  남자를\n찾기는  어렵다 .[4]\n2. 어린 여성의  경우에는 , 대학  졸업을  기점으로  촌티를  벗고  외모의  급격한  향상이  이루어지는  경우가  많다 . 나이가  든  이후\n에도 수험생활이  길거나 , 학위  취득에  전념하다보니  외모관리에  소홀해지는  경우도  흔하다 . 그런데 , 소개팅  시장에  주로\n나오는  여성들은  주로  이런  부류의  사람들이다 . 따라서  처음  보이는  인상만으로  실제로  그  여성이  계속  매력이  없을  것이\n라고 생각할  수  없다 .\n3. 소개팅  외에  이성을  만날  여건  자체가  안  되는  경우가  많다 . 직장인의  경우  주변에  마음에  들지  않는  이성  밖에  없으며  사\n내 연애를 부담스러워  하는  경우에는  주변에  사실상  연애를  할  사람이  없는  셈이다 . 이  경우  사실상  소개팅  외에는  이성을\n만날 기회를  얻기  어렵다 .[5]\n4. 동호회  등  외부  사교활동으로  만날  수  있지  않느냐는  의견이  있을  수  있지만 , 외부  활동을  별로  좋아하지  않는  성격인  경우\n굳이 시간을  쪼개어  연애를  하기  위해  동아리나  동호회의  부수적  활동까지  하기를  원치는  않는  경우도  적지  않다 . 또한 , 여\n성의 경우  남성이  많은  동호회에  가입할  시  여왕벌 취급을  받는  것에  부담감을  느끼거나  혐오감을  느끼기도  한다 . 반면에\n남성의  경우  여초  동호회에  가입하는  것에  대해  여자를  밝히는  사람인  것처럼  오해를  받는  데  상당한  부담을  느끼기도  한\n다.\n

In [31]:
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=100, chunk_overlap=0, length_function=len)
texts = text_splitter.split_documents(pages)

Created a chunk of size 107, which is longer than the specified 100
Created a chunk of size 118, which is longer than the specified 100
Created a chunk of size 113, which is longer than the specified 100
Created a chunk of size 117, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 118, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100


In [32]:
len(texts)

581

In [35]:
texts[0]

Document(page_content='24. 1. 7. 오후  1:32 소개팅  - 나무위키\nhttps://namu.wiki/w/ 소개팅 1/18\uf3d0소개팅 \n최근 수정  시각 : 2023-12-22 13:06:41', metadata={'source': './data/소개팅_나무위키.pdf', 'page': 0})

In [36]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)

/Users/user/miniconda3/envs/fc-ai-signature/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [37]:
retriever = db.as_retriever(search_kwargs={"k": 10})

In [38]:
rel_docs = retriever.get_relevant_documents("주선자의 역할")

/Users/user/miniconda3/envs/fc-ai-signature/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [39]:
len(rel_docs)

10

In [41]:
rel_docs[0]

Document(page_content='2. 주선자와  그  역할\n과거에는  주선자가  대상  남녀가  만날  장소까지  주선해  주고  잠시  동석하여  각자  소개  및  대화를  잠깐  이끌어  주었다가  자리를', metadata={'source': './data/소개팅_나무위키.pdf', 'page': 1})

In [42]:
rel_docs[:3]

[Document(page_content='2. 주선자와  그  역할\n과거에는  주선자가  대상  남녀가  만날  장소까지  주선해  주고  잠시  동석하여  각자  소개  및  대화를  잠깐  이끌어  주었다가  자리를', metadata={'source': './data/소개팅_나무위키.pdf', 'page': 1}),
 Document(page_content='주선자의  역할이  줄어들다  보니 , 아예  주선자조차  남녀를  둘  다  알지  못하는  한다리  건너식  소개팅  양상도  늘어났다 . 예컨대  A', metadata={'source': './data/소개팅_나무위키.pdf', 'page': 1}),
 Document(page_content='는 주선자  역할인  a 는  A 와  b 만  잘  알  뿐  정작  소개팅  상대녀인  B 에  대한  정보는  한다리  건너서만  아는  식 .', metadata={'source': './data/소개팅_나무위키.pdf', 'page': 1})]

In [43]:
len(rel_docs)

10

In [44]:
print(rel_docs[1].page_content)

주선자의  역할이  줄어들다  보니 , 아예  주선자조차  남녀를  둘  다  알지  못하는  한다리  건너식  소개팅  양상도  늘어났다 . 예컨대  A
